In [1]:
'''%pip install tensorflow==2.2.0
%pip install tensorflow-gpu==2.2.0 '''

'''%pip install --upgrade pip
%pip install --upgrade gast
%pip install --upgrade tensorflow-gpu
%pip install --upgrade tensorboard'''
#%pip install gast==0.3.3
#%pip install tensorflow==2.2.0
#%pip install opencv-python-headless

import pandas as pd
import numpy as np
import shutil
import os
import pydicom as pydicom
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution3D, MaxPooling3D
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, SGD
#from tensorflow.keras.utils import np_utils
from tensorflow.keras import backend as K
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import random
from scipy import ndimage   #Multidimensional image processing

import cv2



In [2]:
############################################# 
#.............ARQUITECTURA...................
#############################################
def get_model(width=128, height=128, depth=45, top='GAP'):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.BatchNormalization()(x)

    if top is 'GMP':
        x = layers.GlobalMaxPooling3D()(x)
        x = layers.Dense(units=512, activation="relu")(x)
        x = layers.Dropout(0.3)(x)
        
    else:
        x = layers.GlobalAveragePooling3D()(x)
        x = layers.Dense(units=512, activation="relu")(x)
        x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


def get_model_anna(width=128, height=128, depth=45, top='GAP'):
    inputs = keras.Input((width, height, depth, 1))
            #initializer = tf.kehist(stack)as.initializers.GlorotNormal()
    
    
    x = layers.Conv3D(filters=16, kernel_size=3,  activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    
    x = layers.Conv3D(filters=32, kernel_size=3,  activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    
    x = layers.Conv3D(filters=64, kernel_size=3,  activation="relu")(x)
    
    
    if top is 'GMP':
        x = layers.GlobalMaxPooling3D()(x)
        '''x = layers.Dense(units=512, activation="relu")(x)
        x = layers.Dropout(0.5)(x)'''
    else:
        x = layers.GlobalAveragePooling3D()(x)
        '''x = layers.Dense(units=512,  activation="relu")(x)
        x = layers.Dropout(0.5)(x)'''

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

############################################# 
#.............OPTIMIZADOR....................
#############################################
def _get_optimizer(optimizer_model, lr_n):
    print("[INFO] OPTIMIZADOR: ", optimizer_model)
    # Initiate the optimizer.
    if optimizer_model == 'Adam':
        opt = Adam(lr_n)

    elif optimizer_model == 'Adagrad':
        opt = Adagrad(lr_n)

    elif optimizer_model == 'Adadelta':
        opt = Adadelta(lr_n)

    elif optimizer_model == 'SGD':
        opt = SGD(lr_n)
    else:
        raise ValueError('Optimizer not implemented yet!')
    return opt

############################################# 
#.........FUNCIÓN ENTRENAMIENTO..............
#############################################
def train_classifier(iepoch, ibatch, ifold, optimizer_model, lr_schedule, train_dataset, label_train, validation_dataset, label_val, itop):
    #model = get_model(width=128, height=128, depth=45, top=itop)
    model = get_model_anna(width=128, height=128, depth=45, top=itop)
    model.summary()
    
    print('[INFO] COMPILAR')
    # Compile model.
    opt=_get_optimizer(optimizer_model, lr_schedule)
    model.compile(
        loss="binary_crossentropy",
        optimizer=opt,
        metrics=["acc"],
    )
    
    # Define callbacks.
    checkpoint_cb = keras.callbacks.ModelCheckpoint(
        "/srv/data/HIC/code/resultados/DICOM/DICOM_top"+str(itop)+"_batch"+str(ibatch)+"_opt"+str(optimizer_model)+"_lr"+str(lr_schedule)+"_kfold"+str(ifold)+".h5", save_best_only=True
    )
    early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

    print(train_dataset.shape)
    print(validation_dataset.shape)
    
    '''for ind in range(0,15):
        sample_stack(train_dataset[ind,:,:,:] , rows=6, cols=6, start_with=0, show_every=1)
        hist(train_dataset[ind,:,:,:])'''
        
    print('[INFO] ENTRENAMIENTO')
    # Train the model, doing validation at the end of each epoch
    model.fit(
        x=train_dataset,
        y=label_train,
        epochs=iepoch,
        batch_size=ibatch,
        validation_data=(validation_dataset,label_val),
        verbose=1,
        callbacks=[checkpoint_cb, early_stopping_cb]
    )    
    
    fig, ax = plt.subplots(1, 2, figsize=(20, 3))
    ax = ax.ravel()
    for i, metric in enumerate(["acc", "loss"]):
        ax[i].plot(model.history.history[metric])
        ax[i].plot(model.history.history["val_" + metric])
        ax[i].set_title("Model {}".format(metric))
        ax[i].set_xlabel("epochs")
        ax[i].set_ylabel(metric)
        ax[i].legend(["train", "val"])
        plt.savefig("/srv/data/HIC/code/resultados/DICOM/LOSS_ACC_top"+str(itop)+"_batch"+str(ibatch)+"_opt"+str(optimizer_model)+"_lr"+str(lr_schedule)+"_kfold"+str(ifold)+".png")
    
    # Clean GPU memory
    del model
    K.clear_session()

In [3]:
############################################# 
#...............PREPROCESADO.................
#############################################

def read_file(filepath):   #Aquí solo define la función. 
    """Read and load volume"""
    # Read file
    #print(filepath)
    much_data = np.load(filepath)
    
    #print(much_data.shape)
    
    return much_data


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 45
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def resize_volume_black(img):    
    n,m,l=img.shape
    #print('img',img.shape)
    img_p=list(img)

    matriz_float = np.zeros((m,l))
    #print('matriz_float',matriz_float.shape)
    
    matriz_float = np.float32(matriz_float)
    for ind in range(n,45):
        img_p.append(matriz_float)
    img_p=np.array(img_p)
    #print('img_nuevo',img_p.shape)
    
    
    n,m,l=img_p.shape
    img_data=np.zeros([128,128,n])
    
    for i in range(0,n):
        img_data[:,:,i]=cv2.resize(img_p[i,:,:],(128,128))
    '''hist(img_data)
    sample_stack(img_data, rows=6, cols=6, start_with=0, show_every=1)'''
    #print(img_data.shape)
    
    return img_data

#COMPENDIO DE LAS FUNCIONES DEFINIDAS ARRIBA:
def process_scan(path):
    """Read and resize volume"""
    #print(path)
    # Read scan
    volume = read_file(path)    #no importan tanto el nombre (path o filepath) sino el ORDEN!!!
    '''hist(volume)
    sample_stack(volume, rows=4, cols=4, start_with=0, show_every=1)'''
    #print('antes',volume.shape)
    
    # Resize width, height and depth
    volume = resize_volume_black(volume)
    #print('despues',volume.shape)
    '''hist(volume)
    sample_stack(volume, rows=6, cols=6, start_with=0, show_every=1)'''
   
    return volume
# VISUALIZAR CORTES
def sample_stack(stack, rows=5, cols=5, start_with=0, show_every=1):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[:,:,ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()
    
# VISUALIZAR CORTES
def hist(stack):
    fig= plt.hist(stack.flatten(), bins=80)
    plt.xlabel("Hounsfield Units (HU)")
    plt.ylabel("Frequency")
    plt.show()

In [4]:

###############################################################
#....INICIALIZAR PARÁMETROS PARA EL ENTRENAMIENTO..............
###############################################################
Kfold=[0,1,2,3,4,5]
batchs=[2] 
epochs = 10
lr=[0.35,0.3]
optimizer_model=['Adadelta'] # 'SGD', 'Adam', 'SGD'
top=['GMP'] #,'GAP'

In [ ]:
import cv2
for ibatch in batchs:
    for ifold in Kfold:
        print('[INFO] FOLD',str(ifold))
        print('\n[INFO] LEYENDO CSV')
        if ifold is not 0:
            CSV_TRAIN = pd.read_csv('/srv/data/HIC/datos/csv/TRAIN'+str(ifold)+'.csv',sep=',')
            CSV_VAL = pd.read_csv('/srv/data/HIC/datos/csv/VAL'+str(ifold)+'.csv',sep=',')
        else:
            CSV_TRAIN = pd.read_csv('/srv/data/HIC/datos/csv/TRAIN.csv',sep=',')
            CSV_VAL = pd.read_csv('/srv/data/HIC/datos/csv/TEST.csv',sep=',')
        train_dataset=[]
        for i in CSV_TRAIN['name']:
            #print(i)
            much_data= process_scan(i)
            #print(much_data.shape)
            train_dataset.append(much_data)         
        train_dataset=np.array(train_dataset)
        x_train = np.expand_dims(train_dataset, axis=4)
        y_train=CSV_TRAIN['label']
        
        validation_dataset=[]
        for i in CSV_VAL['name']:
            much_data= process_scan(i)
            validation_dataset.append(much_data)
            validation_dataset=np.array(validation_dataset)
            validation_dataset = validation_dataset.astype(np.float32)
            validation_dataset=list(validation_dataset)

        validation_dataset=np.array(validation_dataset)
        x_val= np.expand_dims(validation_dataset, axis=4)
        y_val=CSV_VAL['label']
        
        x_train = np.asarray(x_train)
        y_train = np.asarray(y_train)
        x_val = np.asarray(x_val)
        y_val = np.asarray(y_val)
        print(
            "Number of samples in train and validation are %d and %d." 
            % (x_train.shape[0], x_val.shape[0])
        )
        '''# Define data loaders.
        train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
        print(train_loader)
    
        batch_size = ibatch
        print('[INFO] BATCH',str(ibatch))
        
        # Augment the on the fly during training.
        train_dataset = (
            train_loader.shuffle(len(x_train))
            .map(train_preprocessing)
            .batch(batch_size)
            .prefetch(2)
        )
        # Only rescale.
        validation_dataset = (
            validation_loader.shuffle(len(x_val))
            .map(validation_preprocessing)
            .batch(batch_size)
            .prefetch(2)
        )'''
        for ilr in lr:
            '''initial_learning_rate = ilr_p
            lr_schedule = keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
            )
            ilr=lr_schedule'''
            for iopt in optimizer_model:
                for itop in top:
                    print('[INFO] BATCH',str(ibatch))           
                    print('[INFO] FOLD',str(ifold))
                    print('[INFO] OPT',str(iopt))
                    print('[INFO] LR',str(ilr))
                    print('[INFO] topmodel',str(itop))
                    
                    '''print('\n[INFO] LEYENDO CSV')
                    CSV_TRAIN = pd.read_csv('/srv/data/HIC/datos/csv/TRAIN'+str(ifold)+'.csv',sep=',')
                    CSV_VAL = pd.read_csv('/srv/data/HIC/datos/csv/VAL'+str(ifold)+'.csv',sep=',')

                    train_dataset=[]
                    for i in CSV_TRAIN['name']:
                        much_data = np.load(i)
                        much_data.resize((45, 128, 128))
                        much_data=much_data.reshape(much_data.shape[1], much_data.shape[2], much_data.shape[0])

                        #print(much_data.shape)

                        train_dataset.append(much_data)
                        train_dataset=np.array(train_dataset)
                        train_dataset = train_dataset.astype(np.int16)
                        train_dataset=list(train_dataset)
                    train_dataset=np.array(train_dataset)
                    train_dataset = np.expand_dims(train_dataset, axis=4)
                    label_train=CSV_TRAIN['label']

                    validation_dataset=[]
                    for i in CSV_VAL['name']:
                        much_data = np.load(i)
                        much_data.resize((45, 128, 128))
                        much_data=much_data.reshape(much_data.shape[1], much_data.shape[2], much_data.shape[0])

                        validation_dataset.append(much_data)
                        validation_dataset=np.array(validation_dataset)
                        validation_dataset = validation_dataset.astype(np.int16)
                        validation_dataset=list(validation_dataset)

                    validation_dataset=np.array(validation_dataset)
                    validation_dataset= np.expand_dims(validation_dataset, axis=4)
                    label_val=CSV_VAL['label']'''

                    #Entrenar
                    train_classifier(epochs, ibatch, ifold, iopt, ilr, x_train, y_train, x_val, y_val, itop)

[INFO] FOLD 0

[INFO] LEYENDO CSV
Number of samples in train and validation are 165 and 42.
[INFO] BATCH 2
[INFO] FOLD 0
[INFO] OPT Adadelta
[INFO] LR 0.35
[INFO] topmodel GMP
Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 45, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 126, 126, 43, 16)  448       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 63, 63, 21, 16)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 61, 61, 19, 32)    13856     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 30, 9, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)

In [139]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from tensorflow.keras.utils import to_categorical
from tabulate import tabulate
def plot_confusion_matrix(y_true, y_pred, numfold, classes, normalize=False, title=None, cmap=plt.cm.Blues, dir_out='/srv/data/HIC/code/resultados/DICOM/'):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    # classes = [classes['0'], classes['1'], classes['2'], classes['3']]
    # classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    ax.figure.savefig(dir_out + '/kfold' + str(numfold))
    plt.close()

    return ax


def report_classification_results(y_true, y_pred, classes, numfold, paths='/srv/data/HIC/code/resultados/DICOM/'):
    # Extract confusion matrix
    conf_mat = metrics.confusion_matrix(y_true, y_pred)

    # Compute metrics
    headers = []
    g_TP, g_TN, g_FN, g_FP = [], [], [], []
    ss, ee, pp, nn, ff, aa = [], [], [], [], [], []
    S, E, PPV, NPV, F, ACC = ['Sensitivity'], ['Specificity'], ['PPV (precision)'], ['NPV'], ['F1-score'], ['Accuracy']
    for i in range(len(classes)):
        headers.append(classes[str(i)])

        # Extract indicators per class
        TP = conf_mat[i, i]
        TN = sum(sum(conf_mat)) - sum(conf_mat[i, :]) - sum(conf_mat[:, i]) + TP
        FN = sum(conf_mat[i, :]) - conf_mat[i, i]
        FP = sum(conf_mat[:, i]) - conf_mat[i, i]

        # Extract metrics per class
        sen = TP / (TP + FN)
        spe = TN / (TN + FP)
        ppv = TP / (TP + FP)
        npv = TN / (TN + FN)
        f1_s = 2 * ppv * sen / (ppv + sen)
        acc = (TP + TN) / (TP + TN + FP + FN)

        # Create table for printing
        S.append(str(round(sen, 4)))
        E.append(str(round(spe, 4)))
        PPV.append(str(round(ppv, 4)))
        NPV.append(str(round(npv, 4)))
        F.append(str(round(f1_s, 4)))
        ACC.append(str(round(acc, 4)))

        # Global indicators
        g_TP.append(TP), g_TN.append(TN), g_FP.append(FP), g_FN.append(FN)
        # Global metrics
        ss.append(sen), ee.append(spe), pp.append(ppv), nn.append(npv), ff.append(f1_s), aa.append(acc)

    # Extract micro values
    micro_S = round(sum(g_TP) / (sum(g_TP) + sum(g_FN)), 4)
    micro_E = round(sum(g_TN) / (sum(g_TN) + sum(g_FP)), 4)
    micro_PPV = round(sum(g_TP) / (sum(g_TP) + sum(g_FP)), 4)
    micro_NPV = round(sum(g_TN) / (sum(g_TN) + sum(g_FN)), 4)
    micro_F1 = round(2 * micro_S * micro_PPV / (micro_S + micro_PPV), 4)
    micro_Acc = round((sum(g_TP) + sum(g_TN)) / (sum(g_TP) + sum(g_TN) + sum(g_FP) + sum(g_FN)), 4)
    # Extract macro values
    macro_S = round(sum(ss) / len(ss), 4)
    macro_E = round(sum(ee) / len(ee), 4)
    macro_PPV = round(sum(pp) / len(pp), 4)
    macro_NPV = round(sum(nn) / len(nn), 4)
    macro_F1 = round(sum(ff) / len(ff), 4)
    macro_Acc = round(sum(aa) / len(aa), 4)

    # Construct the table
    S.append('---'), E.append('---'), PPV.append('---'), NPV.append('---'), F.append('---'), ACC.append('---')
    S.append(micro_S), E.append(micro_E), PPV.append(micro_PPV), NPV.append(micro_NPV), F.append(micro_F1), ACC.append(
        micro_Acc)
    S.append(macro_S), E.append(macro_E), PPV.append(macro_PPV), NPV.append(macro_NPV), F.append(macro_F1), ACC.append(
        macro_Acc)

    # Define table
    my_data = [tuple(S), tuple(E), tuple(PPV), tuple(NPV), tuple(F), tuple(ACC)]
    df = pd.DataFrame(my_data)
    df.to_csv(paths + '/metrics_kfold' + str(numfold) + '.csv')

    # MODEL metrics
    if len(classes) > 2:
        auc = metrics.roc_auc_score(y_true=to_categorical(y_true, num_classes=len(classes)),
                                    y_score=to_categorical(y_pred, num_classes=len(classes)), multi_class='ovr')
    else:
        auc = metrics.roc_auc_score(y_true=to_categorical(y_true, num_classes=2),
                                    y_score=to_categorical(y_pred, num_classes=2))
    # Printing results
    headers.append('-'), headers.append('micro-Avg'), headers.append('macro-Avg')
    print(tabulate(my_data, headers=headers))

    print('------------------------\nAUC', round(auc, 4))

    # Confusion matrix
    print('Plot confusion_matrix:')
    plot_confusion_matrix(y_true, y_pred, numfold, classes, dir_out=paths)

    # Coeficiente de cohen Kappa
    print('Coeficiente de cohen Kappa:')
    kappa = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    print(kappa)

    # print('Plot curve ROC:')
    # curve_ROC(len(classes),y_true,y_pred,numfold,dir_out=paths['folder_to_saveMetrics'])
    return micro_S

In [140]:

# Load best weights.
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Layer, InputSpec
from sklearn import metrics


for iifold in range(0,6):
    model=load_model('/srv/data/HIC/code/resultados/DICOM/DICOM_topGMP_batch2_optAdadelta_lr0.35_kfold'+str(iifold)+'.h5')
    #model.load_weights("/srv/data/HIC/code/resultados/DICOM/DICOM_topGMP_batch2_optAdadelta_lr0.01_kfold1.h5")
    x_test=[]
    CSV_TEST = pd.read_csv('/srv/data/HIC/datos/csv/TEST.csv',sep=',')
    test_dataset=[]
    for i in CSV_TEST['name']:
        much_data= process_scan(i)  
        test_dataset.append(much_data)         
    x_test=np.array(test_dataset)
    x_test = np.expand_dims(x_test, axis=4)
    y_test=CSV_TEST['label']

    y_pred = model.predict(x_test)  
    #print(y_pred.round())
    y_pred=y_pred.round()
    prediction = y_pred[0]
    scores = [1 - prediction[0], prediction[0]]
    print()
    class_names = ["NORMAL", "HIC"]
    for score, name in zip(scores, class_names):
        print(
            "This model is %.2f percent confident that CT scan is %s"
            % ((100 * score), name)
        )


    classes = {'0': 'Sano', '1': 'HIC'}
    plot_confusion_matrix(y_test, y_pred, iifold, classes)
    report_classification_results(y_test, y_pred, classes, iifold)



This model is 0.00 percent confident that CT scan is NORMAL
This model is 100.00 percent confident that CT scan is HIC
Confusion matrix, without normalization
[[26  0]
 [ 0 16]]
                   Sano    HIC  -      micro-Avg    macro-Avg
---------------  ------  -----  ---  -----------  -----------
Sensitivity           1      1  ---            1            1
Specificity           1      1  ---            1            1
PPV (precision)       1      1  ---            1            1
NPV                   1      1  ---            1            1
F1-score              1      1  ---            1            1
Accuracy              1      1  ---            1            1
------------------------
AUC 1.0
Plot confusion_matrix:
Confusion matrix, without normalization
[[26  0]
 [ 0 16]]
Coeficiente de cohen Kappa:
1.0

This model is 0.00 percent confident that CT scan is NORMAL
This model is 100.00 percent confident that CT scan is HIC
Confusion matrix, without normalization
[[26  0]
 [ 0 16]]